# Automated Lateral Digitization and Meter Association

## Introduction
This notebook outlines a method for generating approximately 300,000 service lateral lines from existing lateral line features and associating them with nearby water meters. The purpose of this workflow is to automate the digitization of lateral features to:

- Enable the generation of unique identifiers that can be used to track work orders related to individual meters.
- Associate laterals with meter locations to determine unknown lateral attributes.
- Provide necessary infrastructure data to support EPA Lead and Copper Rule compliance, including material inventory and risk identification.

## Overview of Process
 
1. Subset data using a grid index to optimize performance.
2. For each lateral line feature:
     - Generate evenly spaced vertices along the line.
     - Convert those vertices to points.
3. For each point:
     - Use a proximity analysis to find the nearest water meter.
     - Retrieve meter attributes such as Meter ID.
4. For each vertex-meter pair:
     - Extract XY coordinates.
     - Generate a line feature connecting them.
5. Compile all outputs into a new feature class of digitized laterals with associated metadata.

In [ ]:

import arcpy
import os

# Set environment
arcpy.env.overwriteOutput = True

# Input parameters
workspace = "C:/GISProject/"
grid_index = os.path.join(workspace, "grid_index.shp")
laterals = os.path.join(workspace, "lateral_lines.shp")
meters = os.path.join(workspace, "water_meters.shp")
out_lateral_points = os.path.join(workspace, "lateral_vertices.shp")
out_lines = os.path.join(workspace, "digitized_laterals.shp")
temp_near_table = os.path.join(workspace, "near_table.dbf")

# Create evenly spaced points along each lateral
points_along_lines = os.path.join(workspace, "points_along_lines.shp")
arcpy.management.GeneratePointsAlongLines(
    Input_Features=laterals,
    Output_Feature_Class=points_along_lines,
    Distance="2 Meters",
    Include_End_Points="NO_END_POINTS"
)

# Use spatial index to process in chunks
with arcpy.da.SearchCursor(grid_index, ["SHAPE@", "ID"]) as grid_cursor:
    for grid_row in grid_cursor:
        grid_geom = grid_row[0]
        grid_id = grid_row[1]

        # Select points in the current grid cell
        temp_points = os.path.join(workspace, f"temp_points_{grid_id}.shp")
        arcpy.analysis.Clip(points_along_lines, grid_geom, temp_points)

        # Generate near table from points to meters
        arcpy.analysis.GenerateNearTable(
            in_features=temp_points,
            near_features=meters,
            out_table=temp_near_table,
            search_radius="50 Meters",
            location="LOCATION",
            angle="NO_ANGLE",
            closest="CLOSEST",
            closest_count=1
        )

        # Join near table to get meter attributes
        arcpy.management.JoinField(
            in_data=temp_points,
            in_field="FID",
            join_table=temp_near_table,
            join_field="IN_FID",
            fields=["NEAR_FID", "NEAR_X", "NEAR_Y"]
        )

        # Add XY for lateral point
        arcpy.management.AddGeometryAttributes(temp_points, "POINT_X_Y")

        # Create line features between lateral vertex and matched meter
        array = arcpy.Array()
        spatial_ref = arcpy.Describe(temp_points).spatialReference
        line_schema = ["POINT_X", "POINT_Y", "NEAR_X", "NEAR_Y"]

        with arcpy.da.SearchCursor(temp_points, line_schema) as cursor, \
             arcpy.da.InsertCursor(out_lines, ["SHAPE@"])
        :
            for row in cursor:
                array.removeAll()
                start = arcpy.Point(row[0], row[1])
                end = arcpy.Point(row[2], row[3])
                array.add(start)
                array.add(end)
                polyline = arcpy.Polyline(array, spatial_ref)
                _.insertRow([polyline])

print("Digitization process complete. Output saved to:", out_lines)